In [1]:
from pprint import pprint
import qlib
import pandas as pd
from qlib.utils.time import Freq
from qlib.utils import flatten_dict
from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.evaluate import risk_analysis
from qlib.contrib.strategy import TopkDropoutStrategy
from qlib.utils import init_instance_by_config

from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord

from qlib.contrib.report import analysis_model, analysis_position

from qlib.backtest import backtest, executor,exchange


In [2]:

qlib.init(provider_uri='~/.qlib/qlib_data/ch_data')

# MARKET = "all"
# BENCHMARK = "000300"
# EXP_NAME = "tutorial_exp"
# CSI300_BENCH = "000300"

[23464:MainThread](2024-04-08 21:27:28,940) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[23464:MainThread](2024-04-08 21:27:28,952) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[23464:MainThread](2024-04-08 21:27:28,954) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/Lincoln/.qlib/qlib_data/ch_data')}


In [3]:
# 数据参数
handler_kwargs = {
        "start_time": "2021-02-01",
        "end_time": "2023-12-29",
        # "fit_start_time": "2020-01-01",
        # "fit_end_time": "2022-05-31",
        "instruments": 'all',
}

# 因子生成参数
handler_conf = {
    "class": "Alpha158",
    "module_path": "qlib.contrib.data.handler",
    "kwargs": handler_kwargs,
}

pprint(handler_conf)

{'class': 'Alpha158',
 'kwargs': {'end_time': '2023-12-29',
            'instruments': 'all',
            'start_time': '2021-02-01'},
 'module_path': 'qlib.contrib.data.handler'}


In [4]:
hd = init_instance_by_config(handler_conf)
df = hd.fetch()
df.head()

[23464:MainThread](2024-04-08 21:53:26,125) INFO - qlib.timer - [log.py:128] - Time cost: 72.882s | Loading data Done
[23464:MainThread](2024-04-08 21:53:26,680) INFO - qlib.timer - [log.py:128] - Time cost: 0.170s | DropnaLabel Done
[23464:MainThread](2024-04-08 21:53:28,389) INFO - qlib.timer - [log.py:128] - Time cost: 1.707s | CSZScoreNorm Done
[23464:MainThread](2024-04-08 21:53:28,405) INFO - qlib.timer - [log.py:128] - Time cost: 2.277s | fit & process data Done
[23464:MainThread](2024-04-08 21:53:28,407) INFO - qlib.timer - [log.py:128] - Time cost: 75.163s | Init data Done


KMID      KLEN     KMID2       KUP      KUP2  \
datetime   instrument                                                     
2021-02-01 000001.SZ   0.067392  0.099562  0.676886  0.019128  0.192124   
           000002.SZ   0.003575  0.030747  0.116282  0.001790  0.058210   
           000063.SZ   0.010632  0.027830  0.382030  0.008754  0.314550   
           000069.SZ   0.013679  0.022793  0.600141  0.004549  0.199574   
           000100.SZ   0.037933  0.067813  0.559382  0.018390  0.271180   

                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...  \
datetime   instrument                                                    ...   
2021-02-01 000001.SZ   0.013042  0.130990  0.061306  0.615752  0.936863  ...   
           000002.SZ   0.025382  0.825508  0.027167  0.883580  0.996437  ...   
           000063.SZ   0.008444  0.303420  0.010322  0.370899  0.989480  ...   
           000069.SZ   0.004565  0.200285  0.013695  0.600852  0.986506  ...   
           000100.SZ   0.011490  0.169438  0.031034  0.457641  0.963453  ...   

                       VSUMN10  VSUMN20  VSUMN30  VSUMN60  VSUMD5  VSUMD10  \
datetime   instrument                                                        
2021-02-01 000001.SZ       NaN      NaN      NaN      NaN     NaN      NaN   
           000002.SZ       NaN      NaN      NaN      NaN     NaN      NaN   
           000063.SZ       NaN      NaN      NaN      NaN     NaN      NaN   
           000069.SZ       NaN      NaN      NaN      NaN     NaN      NaN   
           000100.SZ       NaN      NaN      NaN      NaN     NaN      NaN   

                       VSUMD20  VSUMD30  VSUMD60    LABEL0  
datetime   instrument                                       
2021-02-01 000001.SZ       NaN      NaN      NaN  0.071738  
           000002.SZ       NaN      NaN      NaN -0.014819  
           000063.SZ       NaN      NaN      NaN  0.006936  
           000069.SZ       NaN      NaN      NaN  0.038808  
           000100.SZ       NaN      NaN      NaN -0.025612  

[5 rows x 159 columns]

In [8]:
def factors_process(df):
    # 剔除全部为NA的因子
    df = df.dropna(axis=1, how='all')

    # 剔除行值全相同的列
    cols_to_drop = []
    for col in df.columns:
        if df[col].nunique() == 1:
            cols_to_drop.append(col)
    df = df.drop(cols_to_drop, axis=1)

    return df

In [13]:
df_p = factors_process(df)
outpath = '../量化data/alpha158/'
columns = df_p.columns
for i in columns:
    df_A = df_p[i].unstack()
    df_A = df_A.apply(lambda x: x.fillna(method='ffill'))  # 填补因子缺失值
    df_A.to_csv(outpath + i + '.csv')
    